# Scraping

Désavantages : Long à executer et on ne peut pas récupérer les données du passé


In [3]:
import bs4
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [6]:

page = urllib.request.urlopen("https://www.boursorama.com/cours/AMZN/")

url = "https://www.boursorama.com/cours/"
list_stocks = ["FB", "AAPL", "AMZN", "GOOGL"]


def build_df_scrap(url, list_stocks, nb_max_iteration):
    scrap_dict = {"FB": [], "AAPL": [], "AMZN": [], "GOOGL" : []}
    i=0
    while True and i <= len(list_stocks)*nb_max_iteration : 
        for s in list_stocks :
            link = url+s+"/"
            page = urllib.request.urlopen(link)
            content = page.read().decode('utf-8')
            soup = bs4.BeautifulSoup(content, "html.parser")
            price = soup.find("span", "c-instrument c-instrument--last").decode_contents(formatter="html")
            scrap_dict[s].append(price)
            print("current price of "+ s+ " is : "+ price)
            i+=1
    df = pd.DataFrame(scrap_dict)
    return df

In [7]:
df = build_df_scrap(url, list_stocks, 5)
df.head()

current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80
current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80
current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80
current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80
current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80
current price of FB is : 273.55
current price of AAPL is : 122.41
current price of AMZN is : 3 116.42
current price of GOOGL is : 1 774.80


,FB,AAPL,AMZN,GOOGL
0,273.55,122.41,3 116.42,1 774.80
1,273.55,122.41,3 116.42,1 774.80
2,273.55,122.41,3 116.42,1 774.80
3,273.55,122.41,3 116.42,1 774.80
4,273.55,122.41,3 116.42,1 774.80


# API

Désavantages: l'api key n'est utilisable qu'une fois par jour et pour un nombre de données limité

In [10]:
from alpha_vantage.timeseries import TimeSeries
import json


def save_dataset(symbol):
    # api_key1 = "XNZ8ETR377XWBBNZ" #gmail
    #api_key2 = "SRHMKQTD8VRRDM6G"   #sorbonne 
    #api_key3 = "H6NZR8D6IEPFIBDM" #hotmail
    api_key5 = "123MJ7OCUGMAOWGO" #ensae nb
    #api_key5 = "8HING7T582D8TDSF"
    ts = TimeSeries(key=api_key5, output_format='pandas')
    ts.get_daily(symbol= symbol)
    data, meta_data = ts.get_daily(symbol, outputsize='full')

    data.to_csv(f'./{symbol}_daily.csv')
    


In [11]:
stock_symbole = [ "FB" , "AAPL"]
symbole = stock_symbole[1]
save_dataset(symbole)
df_aapl = pd.read_csv("AAPL_daily.csv")
df_aapl.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2020-12-11,122.43,122.76,120.55,122.41,86939786.0
1,2020-12-10,120.50,123.87,120.15,123.24,81312170.0
2,2020-12-09,124.53,125.95,121.00,121.78,115089193.0
3,2020-12-08,124.37,124.98,123.09,124.38,82225512.0
4,2020-12-07,122.31,124.57,122.25,123.75,86711990.0


# Web

Avantage: utilisation  ilimitée mais moins de données que Alpha_vantage

In [12]:
import pandas_datareader as web
import pandas as pd
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('today')

df_web_aapl = web.DataReader('AAPL', data_source='yahoo', start=start, end=end)[:2156]
df_web_aapl.columns = ['High_APPL', 'Low_APPL', 'Open_APPL', 'Close_APPL', 'Volume_APPL', 'Adj_Close_APPL']
df_web_fb = web.DataReader('FB', data_source='yahoo', start=start, end=end)[:2156]
df_web_fb.columns = ['High_FB', 'Low_FB', 'Open_FB', 'Close_FB', 'Volume_FB', 'Adj_Close_FB']
df_web_amzn = web.DataReader('AMZN', data_source='yahoo', start=start, end=end)[:2156]
df_web_amzn.columns = ['High_AMZN', 'Low_AMZN', 'Open_AMZN', 'Close_AMZN', 'Volume_AMZN', 'Adj_Close_AMZN']
df_web_googl = web.DataReader('GOOGL', data_source='yahoo', start=start, end=end)[:2156]
df_web_googl.columns = ['High_GOOGL', 'Low_GOOGL', 'Open_GOOGL', 'Close_GOOGL', 'Volume_GOOGL', 'Adj_Close_GOOGL']
df_web_vix = web.DataReader('^VIX', data_source='yahoo', start=start, end=end)[:2156]
df_web_vix.columns = ['High_VIX', 'Low_VIX', 'Open_VIX', 'Close_VIX', 'Volume_VIX', 'Adj_Close_VIX']
df_web_vix = df_web_vix[['High_VIX', 'Low_VIX', 'Open_VIX', 'Close_VIX', 'Adj_Close_VIX']]

df_gafa_and_vix =pd.concat([df_web_aapl,df_web_amzn,df_web_googl,df_web_vix], axis=1)  # je mets pas celui de fb psq ca marche pas
df_gafa_and_vix.head()

,High_APPL,Low_APPL,Open_APPL,Close_APPL,Volume_APPL,Adj_Close_APPL,High_AMZN,Low_AMZN,Open_AMZN,Close_AMZN,...,Low_GOOGL,Open_GOOGL,Close_GOOGL,Volume_GOOGL,Adj_Close_GOOGL,High_VIX,Low_VIX,Open_VIX,Close_VIX,Adj_Close_VIX
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,14.732142,14.607142,14.621428,14.686786,302220800.0,12.566676,179.479996,175.550003,175.889999,179.029999,...,326.511505,326.796783,333.038025,7345600.0,333.038025,23.100000,22.540001,22.950001,22.969999,22.969999
2012-01-04,14.810000,14.617143,14.642858,14.765715,260022000.0,12.634213,180.500000,176.070007,179.210007,177.509995,...,330.640656,332.847839,334.474487,5722200.0,334.474487,23.730000,22.219999,23.440001,22.219999,22.219999
2012-01-05,14.948215,14.738214,14.819643,14.929643,271269600.0,12.774481,178.250000,174.050003,175.940002,177.610001,...,328.443451,331.396393,329.834839,6559200.0,329.834839,23.090000,21.340000,22.750000,21.480000,21.480000
2012-01-06,15.098214,14.972143,14.991786,15.085714,318292800.0,12.908023,184.649994,177.500000,178.070007,182.610001,...,325.220215,329.904907,325.335327,5380400.0,325.335327,21.719999,20.580000,21.240000,20.629999,20.629999
2012-01-09,15.276786,15.048214,15.196428,15.061786,394024400.0,12.887549,184.369995,177.000000,182.759995,178.559998,...,310.925934,323.573578,311.541534,11633500.0,311.541534,21.780001,21.000000,21.670000,21.070000,21.070000


In [13]:
df_web_fb.head()

,High_FB,Low_FB,Open_FB,Close_FB,Volume_FB,Adj_Close_FB
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999
